In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from walks import Route, Point, Segment

In [ ]:
import sqlite3
from collections import defaultdict
import statistics

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
conn = sqlite3.connect('mwalks.sqlite')

points = conn.execute("""
    SELECT
        point_id
    ,   route_id
    ,   Cast(strftime('%H', dt) AS INTEGER) * 3600
      + Cast(strftime('%M', dt) AS INTEGER) * 60
      + Cast(strftime('%f', dt) AS REAL) AS secs
    ,   lat
    ,   lon
    ,   ele
    FROM points ORDER BY route_id, point_id
""").fetchall()
print(len(points))

route_bounds = conn.execute('SELECT Min(lon), Min(lat), Max(lon), Max(lat) FROM points').fetchone()
print(route_bounds)

conn.close()

In [ ]:
requested_map = cimgt.OSM()  # Open Street Map

In [ ]:
routes = defaultdict(Route)
for pt in [Point(*p) for p in points]:
    routes[pt.route_id].add_point(pt)
for route_id in range(len(routes) - 5, len(routes) + 1):
    rt = routes[route_id]
    print(f'Route {route_id}: {len(rt.points)} points, distance: {rt.metres() / 1000:.3f}km = {rt.miles():.2f}mi, {rt.time():.2f} sec = {rt.avg_mph():.2f} mph')    


In [ ]:
def locs_at(mins):
    pts = [rt.point_at(mins * 60) for rt in routes.values()]

    lons = [pt.lon for pt in pts if pt]
    lats = [pt.lat for pt in pts if pt]

    return (lons, lats)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=dict(projection=requested_map.crs))

gl = ax.gridlines(draw_labels=True)
gl.top_labels = gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

pad_lat = 0.002
pad_lon = 0.002

lon_min, lat_min, lon_max, lat_max = route_bounds
extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

ax.set_extent(extent)

ax.add_image(requested_map, 14)

lons, lats = locs_at(15)
plt.scatter(lons, lats, color='aqua', s=4, alpha=1, transform=ccrs.PlateCarree())

lons, lats = locs_at(30)
plt.scatter(lons, lats, color='indigo', s=4, alpha=1, transform=ccrs.PlateCarree())

lons, lats = locs_at(45)
plt.scatter(lons, lats, color='red', s=4, alpha=1, transform=ccrs.PlateCarree())

avg_start_lon = statistics.mean(rt.points[0].lon for rt in routes.values())
avg_start_lat = statistics.mean(rt.points[0].lat for rt in routes.values())
plt.scatter([avg_start_lon], [avg_start_lat], color='green', s=30, alpha=1, marker='x', transform=ccrs.PlateCarree())

plt.title(f'After 15, 30, 45 minutes')

plt.show()